# Résumé de la tâche

On a plusieurs éléments sur lesquels travailler à ce stade : 
- un jeu de données contenant des questions et des réponses
- un ensemble de réponses à ces questions
- un corpus traitant de sujets médicaux

Nous disposons également de plusieurs pistes / contraintes :
- une indication du nombre de réponses possibles dans la question (systématique ?)
- la présence ou non d'une négation (on recherche ce qui est vrai ou ce qui est faux ?)
- le vocabulaire médical extrait de chaque question et chaque réponse

# Méthodologie

La première piste serait d'extraire les informations utiles des questions, que nous avons déjà mentionnées :
- nombre de réponses possibles
- négation ou affirmation ?
- vocabulaire médical

Une fois ces informations extraites des questions, on peut "unifier" le vocabulaire de la question avec celui de chaque réponse possible à la question, créant ainsi des ensembles :
- Q1 + R1 = E1
- Q1 + R2 = E2
- Q1 + R3 = E3
- Q1 + R4 = E4

Cela signifie lier un terme ou plusieurs termes médicaux (celui / ceux de la question) avec d'autres termes médicaux (ceux des réponses). Pour chaque ensemble, on regarde s'il existe dans le corpus une phrase (ou autre unité) qui lie les termes de cet ensemble grâce à une mesure de similarité en Ngrammes de char_wb. Une similarité au mot ferait peu de sens dans ce corpus en vue de la singularité des termes employés.

Avec les résultats obtenus, on crée un classement. On regarde ensuite si la question contient une négation et le nombre de réponses indiquées : 
- si on a 3 résultats à trouver, on prend les trois meilleurs ensembles selon les mesures de similarité
- si on a un résultat à trouver et une négation, on prend l'ensemble avec la pire mesure de similarité
- si on a 2 résultats et une négation, on prend les deux pires ensembles en terme de mesure de similarité
- ...

## Extraction du vocabulaire médical

Pour extraire le vocabulaire médical des questions et des réponses, plutôt que de trouver un corpus spécialisé assez exhaustif pour contenir tous les tecnolectes médicaux présents dans notre jeu de données, nous effectuons le raisonnement inverse : nous retrouvons les mots médicaux en supprimant tous les mots n'appartenant pas à ce domaine. Pour chaque question et pour chaque réponse, on a ainsi des listes de tecnolectes médicaux.

## Détection de la négation (pas terminée)

Il existe sûrement une solution avec Spacy (POS tagging -> étiquette "NEG" ?).
Attention : il peut exister des cas où la négation est présente mais n'indique pas une recherche de résultats "faux". 

## Nombre de bons résultats attendus

Voir avec Toufik.

In [2]:
#Imports et fonctions

import re
import json
import pandas as pd
import string as strii

#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise_kernels

import spacy
sp = spacy.load("fr_core_news_sm")

def removePunct(string):
    return string.translate(str.maketrans(strii.punctuation, ' '*len(strii.punctuation))) #map punctuation to space

def tokenizer(string):
    spacy_object = sp(string)
    return [word.text for word in spacy_object if word.is_stop == False] #and word.pos_ != "PUNCT" and word.pos_ != "NUM"

def getKeywords(string,liste_mots,liste_keywords):
    keywords = []
    for word in tokenizer(removePunct(string)):
            word = re.sub(" *","",word)
            if len(word) > 1:
                if word.lower() not in liste_mots:
                    keywords.append(word)
                elif word.lower() in liste_keywords:
                    keywords.append(word)
    return keywords

def cosine_similarity(vec,target,liste):
    return pairwise_kernels(vec.transform([target]),liste,metric='cosine')

def writeJson(path,data):
    with open(path,"w",encoding='utf-8') as f:
        json.dump(data,f,indent=4,ensure_ascii=False)
        
def openJson(path):
    with open(path,'r',encoding='utf-8') as f:
        data = json.load(f)
    return data       

In [ ]:
#Nous avons déjà un corpus de référence. Afin de créer un jeu de données utilisable avec ce corpus,
#nous extrayons de chaque question / réponse son vocabulaire

liste = "input/liste.de.mots.francais.frgut.txt"
with open(liste,'r',encoding='utf-8') as f:
     liste_mots = [line.rstrip('\n').lower() for line in f]
liste_keywords = openJson("output/corpusRef/keywordsMesh.json")
        
df = pd.read_csv("../data/csv/train.csv",delimiter=";")
questions = df["question"]
ids = df["id"]
vocByQuestions = {}

for i in range(len(df)):
    vocByQuestions[ids[i]] = []
    keywords_q = getKeywords(questions[i],liste_mots,liste_keywords)
   
    for element in ["answers."+l for l in "abcde"]:
        keywords = getKeywords(df[element][i],liste_mots,liste_keywords)
        if len(keywords) < 1:
            keywords = ["NULL"]
                    
        vocByQuestions[ids[i]].append(keywords_q+keywords)

writeJson("output/similarités/vocByQuestions.json",vocByQuestions)

In [5]:
#IDEM mais avec la liste de mots de Toufik

liste = "input/listeMotsFR_Auto.txt"
with open(liste,'r',encoding='utf-8') as f:
     liste_mots = [line.rstrip('\n').lower() for line in f]

df = pd.read_csv("../data/csv/train.csv",delimiter=";")
questions = df["question"]
ids = df["id"]
vocByQuestions = {}

for i in range(len(df)):
    vocByQuestions[ids[i]] = []
    keywords_q = getKeywords(questions[i],liste_mots,[])
   
    for element in ["answers."+l for l in "abcde"]:
        keywords = getKeywords(df[element][i],liste_mots,[])
        if len(keywords) < 1:
            keywords = ["NULL"]
                    
        vocByQuestions[ids[i]].append(keywords_q+keywords)

writeJson("output/similarités/vocByQuestions_ListeToufik.json",vocByQuestions)

In [ ]:
#un gros output à la fin / par seuil

dataset = openJson("output/similarités/vocByQuestions.json")
dataref = [" ".join(liste) for liste in openJson("output/corpusRef/manuelMerckSentencesKeywords.json")]

results_principale = []
results_annexe = []

counter = 0
total = len(dataset.keys())

V = CountVectorizer(lowercase=True,ngram_range=(1,3),analyzer="char_wb")
X = V.fit_transform(dataref)
print("Vectorization done")

for key, listes in dataset.items():
    res_principale = {"id":key,"answers":[]}
    res_annexe = {"id":key,"nb_answers":0}
    
    counter += 1
    print(f"{counter}/{total}",end="\r")
    
    for i,question_reponse in enumerate(listes):
        question_reponse = " ".join(question_reponse)
        
        cos = list(cosine_similarity(V,question_reponse,X)[0])
        
        if max(cos) >= 0.7:
            res_principale["answers"].append(str(i))
            res_annexe["nb_answers"] += 1
    
    res_principale["answers"] = "|".join(res_principale["answers"]).replace("0","a").replace("1","b").replace("2","c").replace("3","d").replace("4","e")
    
    results_principale.append(res_principale)
    results_annexe.append(res_annexe)
    
writeJson("output/similarités/taskPrincipale.json",results_principale)
writeJson("output/similarités/taskAnnexe.json",results_annexe)

#garder aussi la similarité
#pas de seuil, conserver les deux meilleures seulement parmi toutes les réponses

In [5]:
#Un output par objet en append / par seuil

dataset = openJson("output/similarités/vocByQuestions.json")
dataref = [" ".join(liste) for liste in openJson("output/corpusRef/manuelMerckSentencesKeywords.json")]

V = CountVectorizer(lowercase=True,ngram_range=(1,3),analyzer="char_wb")
X = V.fit_transform(dataref)
print("Vectorization done")

def writeOutputFile(path,string):
    with open(path, 'a',encoding='utf-8') as f:
        f.write(f"{string}\n")
        
counter = 0
total = len(dataset.keys())

for key, listes in dataset.items():

    good_res = []
    nb_answers = 0
    
    counter += 1
    print(f"{counter}/{total}",end="\r")
    
    for i,question_reponse in enumerate(listes):
        question_reponse = " ".join(question_reponse)
    
        cos = list(cosine_similarity(V,question_reponse,X)[0])

        if max(cos) >= 0.9:
            good_res.append(str(i))
            nb_answers += 1
    
    good_res = "|".join(good_res).replace("0","a").replace("1","b").replace("2","c").replace("3","d").replace("4","e")
    writeOutputFile("output/similarités/taskPrincipale.csv",f"{key};{good_res}")
    writeOutputFile("output/similarités/taskAnnexe.csv",f"{key};{nb_answers}")

Vectorization done


In [4]:
#Un output par objet en append / meilleur résultat seulement

dataset = openJson("output/similarités/vocByQuestions.json")
dataref = [" ".join(liste) for liste in openJson("output/corpusRef/manuelMerckSentencesKeywords.json")]

V = CountVectorizer(lowercase=True,ngram_range=(1,3),analyzer="char_wb")
X = V.fit_transform(dataref)
print("Vectorization done")

def writeOutputFile(path,string):
    with open(path, 'a',encoding='utf-8') as f:
        f.write(f"{string}\n")
        
counter = 0
total = len(dataset.keys())

for key, listes in dataset.items():
    
    good_res = []
    old_maxi = 0
    nb_answers = 0
    
    counter += 1
    print(f"{counter}/{total}",end="\r")
    
    for i,question_reponse in enumerate(listes):
        question_reponse = " ".join(question_reponse)
    
        cos = list(cosine_similarity(V,question_reponse,X)[0])
        maxi = max(cos)

        if maxi > old_maxi:
            old_maxi = maxi
            good_res = [str(i)]
            nb_answers = 1
    
    good_res = good_res[0].replace("0","a").replace("1","b").replace("2","c").replace("3","d").replace("4","e")
    writeOutputFile("output/similarités/taskPrincipale.csv",f"{key};{good_res}")
    writeOutputFile("output/similarités/taskAnnexe.csv",f"{key};{nb_answers}")

Vectorization done


In [ ]:
a = ["pomme banane","pomme","banane","abricot","abricot banane","abricot pomme"]
b = ["pomme banane","pomme","abricot"]
aa = ['tt symptomatique', 'tt symptomatique', '95 normale  sériques transaminases cas cellulaires hépato taux élevés', 'traitement diagnostic Diagnostic biopsie', 'ml sulfate cas kg solution 50  magnésium hypomagnésémie IM', 'acide  repas 300 cp mg dose aspirine acétylsalicylique', 'streptobacillaire pléiomorphe bacille Streptobacillus provoquée  morsure moniliformis fièvre  Gram Fièvre rat', 'SIDA   graves chimiothérapie virus immunodéprimées muqueuse parodonte héréditaires herpès immunosuppresseurs sp Candidose infections bactérie Immunosuppression personnes maladies', ' cumulatif 50 risque', 'traits visage exposées perte soleil vésicules doigts peau zones infection']
bb = ["fausse particules alpha noyaux hélium"]

V = CountVectorizer(lowercase=True,ngram_range=(1,3),analyzer="char_wb")
V.fit(aa+bb)

for c in bb:
    print(pairwise_kernels(V.transform([c]),V.transform(aa),metric='cosine'))
    
[0.4824515401296198, 0.4824515401296198, 0.7745439610488886, 0.522860777712539, 0.7125890792026981, 0.7112519095288717, 0.642633895930556, 0.7074990394260712, 0.6226045694731197, 0.7096781511019711]